In [ ]:
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
import json

import sklearn
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as tk

%matplotlib inline

In [ ]:
IM_DIMS = [256, 256]  

In [ ]:
i = 1


In [ ]:
def resize_points(points, new_dim, old_dim):
    new_points = []
    for point in points:
        new = [float(point[0]) * new_dim[1] / old_dim[1], float(point[1]) * new_dim[0] / old_dim[0] ]
        new_points.append(new)
    return np.array(new_points).astype('int32')
 
def get_file_mask(i):
    im_dir = "./images/imgs_2022/"
    ann_path = f"./images/annotated/im{i}.json" # dirty
    with open(ann_path) as file:
        data = json.load(file)
    points = data['shapes'][0]['points']
    im_path = im_dir + data['imagePath'].split('/')[-1]
    image = cv2.imread(im_path)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    BIG_DIMS = image.shape

    points = resize_points(points, IM_DIMS, BIG_DIMS)


    points = np.array(points).astype('int32')

    
    small_image = cv2.resize(image, [IM_DIMS[1], IM_DIMS[0]])
    bg = np.zeros(small_image.shape[:2])
    bg = cv2.fillPoly(bg, [points], 1)
    bg = bg.astype('int8')
    return small_image, bg

def write_dataset():
    impath = "./images/training/images/"
    lblpath = "./images/training/labels/"
    for i in range(1, 28):
        image, label = get_file_mask(i)
        cv2.imwrite(impath + f"{i}.png", image)
        cv2.imwrite(lblpath + f"{i}.png", label)
        
def get_XY_data():
    X_data = []
    Y_data = []
    for i in range(1,28):
        image, label = get_file_mask(i)
        X_data.append(image)
        Y_data.append(label)
    #X_data = np.expand_dims(X_data, axis=3)
    Y_data = np.expand_dims(Y_data, axis=3)
    return np.array(X_data), np.array(Y_data)
        
        
   

In [ ]:
image, mask = get_file_mask(1)
print(mask.shape)
plt.imshow(image)
plt.show()
plt.imshow(mask, cmap='gray')
plt.show()

In [ ]:
#write_dataset()
X_data, Y_data = get_XY_data()
X_data = X_data.astype("float")/255

In [ ]:
plt.imshow(X_data[5], cmap='gray')
plt.show()
plt.imshow(Y_data[5], cmap='gray')
plt.show()

In [ ]:
# Runtime data augmentation
def get_train_test_augmented(X_data, Y_data, validation_split=0.1, batch_size=32, seed=42):
    X_train, X_test, Y_train, Y_test = train_test_split(X_data,
                                                        Y_data,
                                                        train_size=1-validation_split,
                                                        test_size=validation_split,
                                                        random_state=seed)
    
    # Image data generator distortion options
    data_gen_args = dict(rotation_range=5.,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         shear_range=0.05,
                         zoom_range=0.05,
                         horizontal_flip=False,
                         vertical_flip=False,
                         fill_mode='nearest')  #use 'constant'??


    # Train data, provide the same seed and keyword arguments to the fit and flow methods
    X_datagen = ImageDataGenerator(**data_gen_args)
    Y_datagen = ImageDataGenerator(**data_gen_args)
    X_datagen.fit(X_train, augment=True, seed=seed)
    Y_datagen.fit(Y_train, augment=True, seed=seed)
    X_train_augmented = X_datagen.flow(X_train, batch_size=batch_size, shuffle=True, seed=seed)
    Y_train_augmented = Y_datagen.flow(Y_train, batch_size=batch_size, shuffle=True, seed=seed)
     
    
    # Test data, no data augmentation, but we create a generator anyway
    X_datagen_val = ImageDataGenerator()
    Y_datagen_val = ImageDataGenerator()
    X_datagen_val.fit(X_test, augment=True, seed=seed)
    Y_datagen_val.fit(Y_test, augment=True, seed=seed)
    X_test_augmented = X_datagen_val.flow(X_test, batch_size=batch_size, shuffle=True, seed=seed)
    Y_test_augmented = Y_datagen_val.flow(Y_test, batch_size=batch_size, shuffle=True, seed=seed)
    
    
    # combine generators into one which yields image and masks
    train_generator = zip(X_train_augmented, Y_train_augmented)
    test_generator = zip(X_test_augmented, Y_test_augmented)
    
    return train_generator, test_generator


In [ ]:
# from tensorflow.keras import layers
 
def double_conv_block(x, n_filters):
    # Conv2D then ReLU activation
    x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
    #x = layers.BatchNormalization()(x)
    # Conv2D then ReLU activation
    x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
    #x = layers.BatchNormalization()(x)    
    return x

def downsample_block(x, n_filters):
    f = double_conv_block(x, n_filters)
    p = layers.MaxPool2D(2)(f)
    p = layers.Dropout(0.3)(p)
    return f, p

def upsample_block(x, conv_features, n_filters):
    # upsample
    x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
    # concatenate
    x = layers.concatenate([x, conv_features])
    # dropout
    x = layers.Dropout(0.3)(x)
    # Conv2D twice with ReLU activation
    x = double_conv_block(x, n_filters)

    return x

def get_model():
    inputs = tf.keras.Input(shape=(256, 256, 3))
    # encoder: contracting path - downsample
    # 1 - downsample
    f1, p1 = downsample_block(inputs, 32)
    # 2 - downsample
    f2, p2 = downsample_block(p1, 64)
    # 3 - downsample
    f3, p3 = downsample_block(p2, 128)
    # 4 - downsample
    f4, p4 = downsample_block(p3, 256)
    # 5 - bottleneck
    bottleneck = double_conv_block(p4, 512)
    # decoder: expanding path - upsample
    # 6 - upsample
    u6 = upsample_block(bottleneck, f4, 256)
    # 7 - upsample
    u7 = upsample_block(u6, f3, 128)
    # 8 - upsample
    u8 = upsample_block(u7, f2, 64)
    # 9 - upsample
    u9 = upsample_block(u8, f1, 32)
    # outputs
    outputs = layers.Conv2D(1, 1, padding="same", activation = "softmax")(u9)
    # unet model with Keras Functional API
    unet_model = tf.keras.Model(inputs, outputs, name="U-Net")
    return unet_model




# Free up RAM in case the model definition cells were run multiple times
tf.keras.backend.clear_session()

# Build model
model = get_model()
model.summary()


In [ ]:
#from keras.optimizers import SGD
#opt = SGD(learning_rate=0.01)

# Configure the model for training.
# We use the "sparse" version of categorical_crossentropy
# because our target data is integers.
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics='accuracy')



In [ ]:
# Train the model, doing validation at the end of each epoch.
train_gen, _ = get_train_test_augmented(X_data=X_data,Y_data=Y_data)

X_data_aug = []
Y_data_aug= []

for i in range(100): 
    X_tmp, Y_tmp = next(train_gen)
    X_data_aug.append(X_tmp)
    Y_data_aug.append(Y_tmp)
X_data_aug = np.vstack(X_data_aug)
Y_data_aug = np.vstack(Y_data_aug)

X_data_aug.shape, Y_data_aug.shape


np.savez("data_augmented.npz", X_data=X_data_aug, Y_data=Y_data_aug)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_data_aug,
                                                        Y_data_aug,
                                                        train_size=1-0.1,
                                                        test_size=0.1,
                                                        random_state=42)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape
del(X_data_aug)
del(Y_data_aug)

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("mymodel_augment.h5", save_best_only=True)
]

epochs = 1
model.fit(
    X_train, Y_train,
    batch_size=32,
    epochs=epochs,
    verbose="auto",
    callbacks=callbacks,
    validation_data=(X_test, Y_test),
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)

In [ ]:
model = tf.keras.models.load_model("mymodel.h5")

In [ ]:
preds = model.predict(np.array([X_test[0]]))

In [ ]:
preds.min(), preds.max()

In [ ]:
plt.imshow(preds[0], cmap='gray')
preds.min(), preds.max()
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
bce(Y_train[0:1], preds).numpy()


In [ ]:
plt.imshow(Y_train[0], cmap='gray')

In [ ]:
plt.imshow(X_train[0], cmap='gray')

In [ ]:
im, mask = next(train_gen)
plt.imshow(im[0])
plt.show()
plt.imshow(mask[0], cmap='gray')

Epoch 1/10
68/68 [==============================] - 2195s 32s/step - loss: 0.6877 - accuracy: 0.4563 - val_loss: 0.6782 - val_accuracy: 0.4595
Epoch 2/10
68/68 [==============================] - 2207s 32s/step - loss: 0.7230 - accuracy: 0.4563 - val_loss: 0.6655 - val_accuracy: 0.4595
Epoch 3/10
68/68 [==============================] - 2205s 32s/step - loss: 0.6856 - accuracy: 0.4563 - val_loss: 0.6368 - val_accuracy: 0.4595
Epoch 4/10
68/68 [==============================] - 2204s 32s/step - loss: 0.6088 - accuracy: 0.4563 - val_loss: 0.4325 - val_accuracy: 0.4595
Epoch 5/10
68/68 [==============================] - 2214s 33s/step - loss: 0.4587 - accuracy: 0.4563 - val_loss: 0.3462 - val_accuracy: 0.4595
Epoch 6/10
68/68 [==============================] - 2213s 33s/step - loss: 0.3527 - accuracy: 0.4563 - val_loss: 0.3645 - val_accuracy: 0.4595
Epoch 7/10
68/68 [==============================] - 2208s 32s/step - loss: 0.3332 - accuracy: 0.4563 - val_loss: 0.3218 - val_accuracy: 0.4595
Epoch 8/10
68/68 [==============================] - 2228s 33s/step - loss: 0.3366 - accuracy: 0.4563 - val_loss: 0.2713 - val_accuracy: 0.4595
Epoch 9/10
68/68 [==============================] - 2213s 33s/step - loss: 0.2824 - accuracy: 0.4563 - val_loss: 0.1981 - val_accuracy: 0.4595
Epoch 10/10
68/68 [==============================] - 2210s 33s/step - loss: 0.2114 - accuracy: 0.4563 - val_loss: 0.1495 - val_accuracy: 0.4595

In [ ]:
cv2.imwrite("test.png", image)